# Sparse Autoencoder Training Demo

## Setup

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

from sparse_autoencoder import (
    SparseAutoencoder,
    TensorActivationStore,
    pipeline,
    create_src_dataloader,
)
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from sparse_autoencoder.src_data.datasets.neel_c4_tokenized import (
    collate_neel_c4_tokenized,
)
import torch
import wandb
import os

In [2]:
device = get_device()

### Source Dataset

In [3]:
src_dataloader = create_src_dataloader(
    "NeelNanda/c4-code-tokenized-2b",
    collate_fn=collate_neel_c4_tokenized,
    shuffle_buffer_size=10_000,
    random_seed=0,
    batch_size=64,
)

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

### Source Model

In [4]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype=torch.float32)
src_d_mlp = src_model.cfg.d_mlp
src_d_mlp

Loaded pretrained model solu-1l into HookedTransformer


2048

### Activation Store

In [5]:
max_items = 2_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

### Autoencoder

In [6]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [7]:
wandb.init(project="sparse-autoencoder")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: alan-cooney. Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelis

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x291cabfd0>> (for post_run_cell), with arguments args (<ExecutionResult object at 291ae3250, execution_count=7 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 29fbdc450, raw_cell="wandb.init(project="sparse-autoencoder")" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/alan/Documents/Repos/sparse_autoencoder/demo.ipynb#X20sZmlsZQ%3D%3D> result=<wandb.sdk.wandb_run.Run object at 0x2ccffdbd0>>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [ ]:
pipeline(
    src_model=src_model,
    src_model_activation_hook_point="blocks.0.mlp.hook_post",
    src_model_activation_layer=0,
    src_dataloader=src_dataloader,
    activation_store=store,
    num_activations_before_training=max_items,
    autoencoder=autoencoder,
    device=device,
)

Generate/Train Cycles: 0it [00:00, ?it/s]

Generate Activations:   0%|          | 0/1966080 [00:00<?, ?it/s]

Train Autoencoder:   0%|          | 0/1966080 [00:00<?, ?it/s]

[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 